### Create two empty folders ("html-tables" and "csv-tables") before running this script

In [1]:
import urllib
import numpy as np
import tqdm
from bs4 import BeautifulSoup
import csv
import codecs
import pandas as pd

# Web scraping from demo.istat.it

In [2]:
# From demo.istat.it > Bilancio Demografico Mensile > Vista TERRITORIALE
# Downloads html pages with demographic tables, year 2011 post censimento, it takes 2 minutes
# Ceate an empty folder "html-tables" before running this block
for month in tqdm.tqdm(range(10, 13)):
    for p in range(120):
        url = f'http://demo.istat.it/bilmens20111009/query1.php?lingua=ita&Pro={p}&allrp=4&periodo={month}&submit=Tavola'
        page = urllib.request.urlopen(url)
        written = page.read()
        with open(f'html-tables/2011-{month:02d}-{p:03d}.html', 'wb') as file:
            file.write(written)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:52<00:00, 38.00s/it]


In [3]:
# Downloads html pages with demographic tables, years 2012-2019, it takes roughly one hour
for year in tqdm.tqdm(range(2012, 2020)):
    for month in range(1, 13):
        for p in range(120):
            url = f'http://demo.istat.it/bilmens{year}gen/query1.php?lingua=ita&Pro={p}&allrp=4&periodo={month}&submit=Tavola'
            page = urllib.request.urlopen(url)
            written = page.read()
            with open(f'html-tables/{year}-{month:02d}-{p:03d}.html', 'wb') as file:
                file.write(written)

100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [41:26<00:00, 274.82s/it]


# html to csv

In [4]:
def html2csv(html_path, csv_path):
    try:
        f = codecs.open(html_path, 'r')
        data = f.read()
        soup = BeautifulSoup(data)
        tables = soup.find_all("table")
    except:
        return 2

    if len(tables) < 2:
        return 1
    else:
        table = tables[1] # only the second table is relevant
        output_rows = []
        for table_row in table.findAll('tr'):
            columns = table_row.findAll('td')
            output_row = []
            for column in columns:
                output_row.append(column.text)
            output_rows.append(output_row)

        with open(csv_path, 'a') as f:
            writer = csv.writer(f)
            writer.writerows(output_rows)

        return 0

In [5]:
# Converts html pages in csv tables, it takes 20 minutes
# Create an empty folder "csv-tables" before running this block
for year in tqdm.tqdm(range(2011, 2020)):
    for month in range(1, 13):
        for p in range(120):
            name = f'{year}-{month:02d}-{p:03d}'
            html_path = f'html-tables/{name}.html'
            csv_path = f'csv-tables/{name}.csv'
            result = html2csv(html_path, csv_path)

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [24:13<00:00, 167.43s/it]


# csv big merge

In [6]:
def extract_deaths_comuni(csv_path, year_month):
    state = 'T'
    i = 0
    comuni = []
    try:
        with open(csv_path) as f:
            reader = csv.reader(f)
            for row in reader:
                if len(row) > 0:
                    if row[0] == 'Totale':
                        state = 'T'
                    if row[0] == 'Maschi':
                        state = 'M'
                        i = 0
                    if row[0] == 'Femmine':
                        state = 'F'
                        i = 0
                    if row[0].isdigit():
                        if state == 'T':
                            comuni.append([row[0], row[1], year_month, row[4]])
                        if state == 'M' or state == 'F':
                            comuni[i].append(row[4])
                            i += 1
    except FileNotFoundError:
        pass
    return comuni

In [7]:
# Creates a list of lists, extracting relevant data from csv-tables folder files and merging them
comunilist = []
for year in tqdm.tqdm(range(2011, 2020)):
    for month in range(1,13):
        for p in range(120):
            year_month = f'{year}-{month:02d}'
            csv_path = f'csv-tables/{year_month}-{p:03d}.csv'
            comunilist += extract_deaths_comuni(csv_path, year_month)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:28<00:00,  2.62s/it]


In [8]:
df_comuni = pd.DataFrame(comunilist, columns=['Codice Comune', 'Comuni', 'Mese', 'Morti', 'Maschi', 'Femmine'])
df_comuni.head()

,Codice Comune,Comuni,Mese,Morti,Maschi,Femmine
0,001001,AgliÃ¨,2011-10,6,2,4
1,001002,Airasca,2011-10,4,2,2
2,001003,Ala di Stura,2011-10,1,1,0
3,001004,Albiano d'Ivrea,2011-10,2,2,0
4,001005,Alice Superiore,2011-10,2,1,1


In [9]:
df_comuni.to_csv('morti_comuni_da_2011-10-09_a_2019-11.csv', index=False)